# Stage 1: Generate & Cache Embeddings (Advanced)

**Run this ONCE to pre-compute embeddings, then use Stage 2 for fast experimentation.**

## Features:
- ✅ **Auto-checkpoint**: Saves progress every N batches (resume if interrupted)
- ✅ **Live metrics**: Loss, R², MAE displayed during training
- ✅ **Memory monitoring**: GPU/RAM/Disk usage tracked
- ✅ **Multiple fine-tuning strategies**: Frozen, LoRA, partial unfreeze

## Embedding Strategies:

| Strategy | Trainable Params | Speed | Performance |
|----------|------------------|-------|-------------|
| **A. Frozen** | 0 | Fastest | Baseline |
| **B. LoRA (r=4)** | ~3M (0.25%) | 4.5x faster than full | ~95% of full FT |
| **C. Unfreeze Last 25%** | ~750M (25%) | 30% faster | = Full FT |
| **D. Full Fine-tune** | 3B (100%) | Slowest | Best |

## Time Estimate (A100 40GB, ~140k samples):
- Frozen: ~30-60 min
- LoRA: ~2-3 hours
- Partial unfreeze: ~3-4 hours

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# CONFIGURATION
# ════════════════════════════════════════════════════════════════════════════

CONFIG = {
    # ===== PATHS (MODIFY THESE) =====
    'base_dir': '/content/drive/MyDrive/AbAg_Project',
    'data_file': 'data/ab_ag_affinity_complete.csv',
    'output_dir': 'embeddings',
    'checkpoint_dir': 'checkpoints',
    
    # ===== ENCODER SELECTION =====
    'antibody_encoder': 'IgT5',           # 'IgT5' or 'AbRoBERTa'
    'antigen_encoder': 'ProtT5-Half',     # 'ProtT5-Half', 'ProtT5-Full', 'ESM2'
    
    # ===== FINE-TUNING STRATEGY =====
    # Options: 'frozen', 'lora', 'unfreeze_last_25', 'unfreeze_last_50', 'full'
    'finetune_strategy': 'lora',
    
    # ===== LORA SETTINGS =====
    'lora_r': 4,
    'lora_alpha': 8,
    'lora_dropout': 0.1,
    'lora_target_modules': ['q', 'k', 'v', 'o'],
    
    # ===== LAYER EXTRACTION =====
    'layer_strategy': 'last',  # 'last', 'multi', 'weighted_all'
    'multi_layers': [-1, -12, -18],
    'pooling': 'mean',
    
    # ===== PROCESSING =====
    'batch_size': 4,              # Reduced for fine-tuning (2 large models)
    'gradient_accumulation': 4,   # Effective batch = 4 * 4 = 16
    'max_ab_length': 256,         # Reduced (antibodies are ~150 AA)
    'max_ag_length': 512,         # Reduced (saves memory)
    
    # ===== FINE-TUNING =====
    'ft_epochs': 3,
    'ft_lr': 1e-4,
    'use_gradient_checkpointing': True,  # Saves memory
    
    # ===== CHECKPOINTING =====
    'checkpoint_every_n_batches': 500,  # Save checkpoint every N batches
    'checkpoint_every_n_epochs': 1,     # Save checkpoint every N epochs
    'resume_from_checkpoint': True,     # Auto-resume if checkpoint exists
    
    # ===== MEMORY MANAGEMENT =====
    'clear_cache_every_n_batches': 100,
    'min_disk_space_gb': 5,             # Warn if disk space below this
}

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# SETUP & MEMORY UTILITIES
# ════════════════════════════════════════════════════════════════════════════

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
import gc
import json
import time
import shutil
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm.auto import tqdm
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

# Create directories
BASE_DIR = CONFIG['base_dir']
DATA_PATH = os.path.join(BASE_DIR, CONFIG['data_file'])
OUTPUT_DIR = os.path.join(BASE_DIR, CONFIG['output_dir'])
CHECKPOINT_DIR = os.path.join(BASE_DIR, CONFIG['checkpoint_dir'])

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print(f"Base directory: {BASE_DIR}")
print(f"Data path: {DATA_PATH}")
print(f"Output dir: {OUTPUT_DIR}")
print(f"Checkpoint dir: {CHECKPOINT_DIR}")

In [ ]:
# Install dependencies
!pip install -q transformers>=4.41.0 pandas tqdm sentencepiece peft accelerate psutil
import psutil
print("Dependencies installed!")

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# RESOURCE MONITORING
# ════════════════════════════════════════════════════════════════════════════

class ResourceMonitor:
    """Monitor GPU, RAM, and Disk usage."""
    
    def __init__(self, min_disk_gb=5):
        self.min_disk_gb = min_disk_gb
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    def get_gpu_memory(self):
        """Get GPU memory usage in GB."""
        if torch.cuda.is_available():
            allocated = torch.cuda.memory_allocated() / 1e9
            reserved = torch.cuda.memory_reserved() / 1e9
            total = torch.cuda.get_device_properties(0).total_memory / 1e9
            return {'allocated': allocated, 'reserved': reserved, 'total': total}
        return {'allocated': 0, 'reserved': 0, 'total': 0}
    
    def get_ram_usage(self):
        """Get RAM usage in GB."""
        mem = psutil.virtual_memory()
        return {
            'used': mem.used / 1e9,
            'total': mem.total / 1e9,
            'percent': mem.percent
        }
    
    def get_disk_space(self, path='/content/drive'):
        """Get disk space in GB."""
        try:
            usage = shutil.disk_usage(path)
            return {
                'free': usage.free / 1e9,
                'total': usage.total / 1e9,
                'used': usage.used / 1e9
            }
        except:
            return {'free': 999, 'total': 999, 'used': 0}
    
    def print_status(self, prefix=""):
        """Print current resource status."""
        gpu = self.get_gpu_memory()
        ram = self.get_ram_usage()
        disk = self.get_disk_space()
        
        status = (
            f"{prefix}"
            f"GPU: {gpu['allocated']:.1f}/{gpu['total']:.1f}GB | "
            f"RAM: {ram['used']:.1f}/{ram['total']:.1f}GB ({ram['percent']:.0f}%) | "
            f"Disk: {disk['free']:.1f}GB free"
        )
        print(status)
        
        # Warning if low disk space
        if disk['free'] < self.min_disk_gb:
            print(f"⚠️  WARNING: Low disk space! Only {disk['free']:.1f}GB remaining")
        
        return gpu, ram, disk
    
    def clear_memory(self):
        """Clear GPU and RAM caches."""
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()


# Initialize monitor
monitor = ResourceMonitor(min_disk_gb=CONFIG['min_disk_space_gb'])

# Show initial status
print("="*70)
print("SYSTEM RESOURCES")
print("="*70)
monitor.print_status()

# GPU info
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\nDevice: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Enable optimizations
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# CHECKPOINT MANAGER
# ════════════════════════════════════════════════════════════════════════════

class CheckpointManager:
    """Manage checkpoints for resumable training and embedding generation."""
    
    def __init__(self, checkpoint_dir, config):
        self.checkpoint_dir = checkpoint_dir
        self.config = config
        self.config_str = self._get_config_str()
    
    def _get_config_str(self):
        """Create unique identifier for this configuration."""
        return (
            f"{self.config['antibody_encoder']}_"
            f"{self.config['antigen_encoder']}_"
            f"{self.config['finetune_strategy']}_"
            f"{self.config['layer_strategy']}"
        )
    
    def get_training_checkpoint_path(self):
        return os.path.join(self.checkpoint_dir, f"train_ckpt_{self.config_str}.pth")
    
    def get_embedding_checkpoint_path(self):
        return os.path.join(self.checkpoint_dir, f"embed_ckpt_{self.config_str}.npz")
    
    def save_training_checkpoint(self, epoch, batch_idx, ab_model, ag_model, optimizer, 
                                  train_history, best_metrics):
        """Save training checkpoint."""
        checkpoint = {
            'epoch': epoch,
            'batch_idx': batch_idx,
            'ab_model_state': ab_model.state_dict(),
            'ag_model_state': ag_model.state_dict(),
            'optimizer_state': optimizer.state_dict(),
            'train_history': train_history,
            'best_metrics': best_metrics,
            'config': self.config,
            'timestamp': datetime.now().isoformat(),
        }
        path = self.get_training_checkpoint_path()
        torch.save(checkpoint, path)
        print(f"💾 Checkpoint saved: epoch {epoch+1}, batch {batch_idx}")
        return path
    
    def load_training_checkpoint(self, ab_model, ag_model, optimizer):
        """Load training checkpoint if exists."""
        path = self.get_training_checkpoint_path()
        if os.path.exists(path):
            # PyTorch 2.6+ requires weights_only=False for checkpoints with numpy/custom objects
            checkpoint = torch.load(path, weights_only=False)
            ab_model.load_state_dict(checkpoint['ab_model_state'])
            ag_model.load_state_dict(checkpoint['ag_model_state'])
            optimizer.load_state_dict(checkpoint['optimizer_state'])
            print(f"✅ Resumed from checkpoint: epoch {checkpoint['epoch']+1}, batch {checkpoint['batch_idx']}")
            return checkpoint
        return None
    
    def save_embedding_checkpoint(self, processed_idx, ab_embeddings, ag_embeddings, labels):
        """Save embedding generation progress."""
        path = self.get_embedding_checkpoint_path()
        np.savez_compressed(
            path,
            processed_idx=processed_idx,
            ab_embeddings=ab_embeddings[:processed_idx],
            ag_embeddings=ag_embeddings[:processed_idx],
            labels=labels[:processed_idx],
        )
        print(f"💾 Embedding checkpoint saved: {processed_idx:,} samples")
    
    def load_embedding_checkpoint(self):
        """Load embedding checkpoint if exists."""
        path = self.get_embedding_checkpoint_path()
        if os.path.exists(path):
            data = np.load(path)
            print(f"✅ Resumed embedding from checkpoint: {data['processed_idx']:,} samples done")
            return {
                'processed_idx': int(data['processed_idx']),
                'ab_embeddings': data['ab_embeddings'],
                'ag_embeddings': data['ag_embeddings'],
                'labels': data['labels'],
            }
        return None
    
    def clear_checkpoints(self):
        """Remove all checkpoints for this config."""
        for path in [self.get_training_checkpoint_path(), self.get_embedding_checkpoint_path()]:
            if os.path.exists(path):
                os.remove(path)
                print(f"🗑️  Removed: {path}")


# Initialize checkpoint manager
ckpt_manager = CheckpointManager(CHECKPOINT_DIR, CONFIG)
print(f"Checkpoint config: {ckpt_manager.config_str}")

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# TRAINING LOGGER
# ════════════════════════════════════════════════════════════════════════════

class TrainingLogger:
    """Track and display training metrics."""
    
    def __init__(self):
        self.history = {
            'train_loss': [],
            'val_loss': [],
            'val_r2': [],
            'val_mae': [],
            'epoch_times': [],
        }
        self.current_epoch_losses = []
        self.epoch_start_time = None
    
    def start_epoch(self):
        self.current_epoch_losses = []
        self.epoch_start_time = time.time()
    
    def log_batch(self, loss):
        self.current_epoch_losses.append(loss)
    
    def end_epoch(self, val_loss=None, val_r2=None, val_mae=None):
        epoch_time = time.time() - self.epoch_start_time
        train_loss = np.mean(self.current_epoch_losses) if self.current_epoch_losses else 0
        
        self.history['train_loss'].append(train_loss)
        self.history['epoch_times'].append(epoch_time)
        
        if val_loss is not None:
            self.history['val_loss'].append(val_loss)
            self.history['val_r2'].append(val_r2)
            self.history['val_mae'].append(val_mae)
        
        return train_loss, epoch_time
    
    def get_progress_str(self, epoch, total_epochs, batch_idx=None, total_batches=None):
        """Get formatted progress string."""
        if self.current_epoch_losses:
            recent_loss = np.mean(self.current_epoch_losses[-10:])  # Last 10 batches
        else:
            recent_loss = 0
        
        parts = [f"Epoch {epoch+1}/{total_epochs}"]
        if batch_idx is not None:
            parts.append(f"Batch {batch_idx}/{total_batches}")
        parts.append(f"Loss: {recent_loss:.4f}")
        
        if self.history['val_r2']:
            parts.append(f"Val R²: {self.history['val_r2'][-1]:.4f}")
        
        return " | ".join(parts)
    
    def print_epoch_summary(self, epoch):
        """Print summary for completed epoch."""
        print("\n" + "-"*60)
        print(f"EPOCH {epoch+1} SUMMARY")
        print("-"*60)
        print(f"  Train Loss: {self.history['train_loss'][-1]:.4f}")
        if self.history['val_loss']:
            print(f"  Val Loss:   {self.history['val_loss'][-1]:.4f}")
            print(f"  Val R²:     {self.history['val_r2'][-1]:.4f}")
            print(f"  Val MAE:    {self.history['val_mae'][-1]:.4f}")
        print(f"  Time:       {self.history['epoch_times'][-1]/60:.1f} min")
        print("-"*60)


print("Training logger ready!")

## Load Dataset

In [ ]:
# Load dataset
print("="*70)
print("LOADING DATASET")
print("="*70)

df = pd.read_csv(DATA_PATH)
print(f"Loaded {len(df):,} samples")

# Filter valid pKd range
df = df[(df['pKd'] >= 4.0) & (df['pKd'] <= 14.0)].reset_index(drop=True)
print(f"After filtering (pKd 4-14): {len(df):,} samples")

# Show distribution
print(f"\npKd statistics:")
print(f"  Min: {df['pKd'].min():.2f}")
print(f"  Max: {df['pKd'].max():.2f}")
print(f"  Mean: {df['pKd'].mean():.2f}")
print(f"  Std: {df['pKd'].std():.2f}")

# Split for fine-tuning
if CONFIG['finetune_strategy'] != 'frozen':
    train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
    print(f"\nTrain/Val split: {len(train_df):,} / {len(val_df):,}")
else:
    train_df = df
    val_df = None
    print("\nStrategy is FROZEN - no train/val split needed")

## Fine-Tuning Utilities

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# FINE-TUNING UTILITIES
# ════════════════════════════════════════════════════════════════════════════

def apply_lora(model, config):
    """Apply LoRA adapters to model."""
    from peft import LoraConfig, get_peft_model, TaskType
    
    lora_config = LoraConfig(
        r=config['lora_r'],
        lora_alpha=config['lora_alpha'],
        lora_dropout=config['lora_dropout'],
        target_modules=config['lora_target_modules'],
        bias="none",
        task_type=TaskType.FEATURE_EXTRACTION,
    )
    
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
    return model


def unfreeze_last_n_percent(model, percent=25):
    """Unfreeze last N% of encoder blocks."""
    for param in model.parameters():
        param.requires_grad = False
    
    if hasattr(model, 'encoder') and hasattr(model.encoder, 'block'):
        blocks = model.encoder.block
        total_blocks = len(blocks)
        n_unfreeze = max(1, int(total_blocks * percent / 100))
        
        print(f"  Total blocks: {total_blocks}, Unfreezing last {n_unfreeze} ({percent}%)")
        
        for i in range(total_blocks - n_unfreeze, total_blocks):
            for param in blocks[i].parameters():
                param.requires_grad = True
        
        if hasattr(model.encoder, 'final_layer_norm'):
            for param in model.encoder.final_layer_norm.parameters():
                param.requires_grad = True
    
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"  Trainable: {trainable/1e6:.1f}M / {total/1e6:.1f}M ({100*trainable/total:.1f}%)")
    return model


def apply_finetune_strategy(model, strategy, config):
    """Apply fine-tuning strategy."""
    print(f"  Strategy: {strategy.upper()}")
    
    if strategy == 'frozen':
        for param in model.parameters():
            param.requires_grad = False
    elif strategy == 'lora':
        model = apply_lora(model, config)
    elif strategy == 'unfreeze_last_25':
        model = unfreeze_last_n_percent(model, 25)
    elif strategy == 'unfreeze_last_50':
        model = unfreeze_last_n_percent(model, 50)
    elif strategy == 'full':
        trainable = sum(p.numel() for p in model.parameters())
        print(f"  Trainable: {trainable/1e6:.1f}M (100%)")
    
    return model


# Layer extraction functions
def extract_embeddings_multilayer(hidden_states, attention_mask, layer_indices, pooling='mean'):
    embeddings_list = []
    for idx in layer_indices:
        hidden = hidden_states[idx]
        if pooling == 'mean':
            mask = attention_mask.unsqueeze(-1).float()
            emb = (hidden * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1e-9)
        elif pooling == 'cls':
            emb = hidden[:, 0, :]
        elif pooling == 'max':
            emb = hidden.max(dim=1).values
        embeddings_list.append(emb)
    return torch.cat(embeddings_list, dim=-1)


def extract_embeddings_weighted(hidden_states, attention_mask, pooling='mean'):
    n_layers = len(hidden_states)
    weights = torch.arange(1, n_layers + 1, dtype=torch.float32, device=hidden_states[0].device)
    weights = weights / weights.sum()
    
    weighted_sum = None
    for i, hidden in enumerate(hidden_states):
        if pooling == 'mean':
            mask = attention_mask.unsqueeze(-1).float()
            emb = (hidden * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1e-9)
        elif pooling == 'cls':
            emb = hidden[:, 0, :]
        else:
            emb = hidden.max(dim=1).values
        
        if weighted_sum is None:
            weighted_sum = weights[i] * emb
        else:
            weighted_sum += weights[i] * emb
    return weighted_sum


print("Fine-tuning utilities ready!")

## Load Encoders

In [ ]:
from transformers import T5EncoderModel, T5Tokenizer

ENCODER_MODELS = {
    'IgT5': 'Exscientia/IgT5',
    'AbRoBERTa': 'mogam-ai/Ab-RoBERTa',
    'ProtT5-Half': 'Rostlab/prot_t5_xl_half_uniref50-enc',
    'ProtT5-Full': 'Rostlab/prot_t5_xl_uniref50',
    'ESM2': 'facebook/esm2_t33_650M_UR50D',
}

print("="*70)
print("LOADING ENCODERS")
print("="*70)

# Antibody encoder
ab_model_name = ENCODER_MODELS[CONFIG['antibody_encoder']]
print(f"\n[Antibody] {ab_model_name}")
ab_tokenizer = T5Tokenizer.from_pretrained(ab_model_name)
ab_model = T5EncoderModel.from_pretrained(
    ab_model_name,
    torch_dtype=torch.float16,
    output_hidden_states=True
)

# Enable gradient checkpointing to save memory
if CONFIG.get('use_gradient_checkpointing', False) and CONFIG['finetune_strategy'] != 'frozen':
    ab_model.gradient_checkpointing_enable()
    print("  ✓ Gradient checkpointing enabled")

ab_model = apply_finetune_strategy(ab_model, CONFIG['finetune_strategy'], CONFIG)
ab_model = ab_model.to(device)

# Antigen encoder
ag_model_name = ENCODER_MODELS[CONFIG['antigen_encoder']]
print(f"\n[Antigen] {ag_model_name}")
ag_tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_uniref50')
ag_model = T5EncoderModel.from_pretrained(
    ag_model_name,
    torch_dtype=torch.float16,
    output_hidden_states=True
)

# Enable gradient checkpointing to save memory
if CONFIG.get('use_gradient_checkpointing', False) and CONFIG['finetune_strategy'] != 'frozen':
    ag_model.gradient_checkpointing_enable()
    print("  ✓ Gradient checkpointing enabled")

ag_model = apply_finetune_strategy(ag_model, CONFIG['finetune_strategy'], CONFIG)
ag_model = ag_model.to(device)

# Calculate dimensions
ab_dim = 1024
ag_dim = 1024
if CONFIG['layer_strategy'] == 'multi':
    n_layers = len(CONFIG['multi_layers'])
    ab_output_dim = ab_dim * n_layers
    ag_output_dim = ag_dim * n_layers
else:
    ab_output_dim = ab_dim
    ag_output_dim = ag_dim

print(f"\nOutput dimensions: Ab={ab_output_dim}, Ag={ag_output_dim}, Total={ab_output_dim + ag_output_dim}")
monitor.print_status("\nAfter loading: ")

## Fine-Tuning (with checkpoints)

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# FINE-TUNING WITH CHECKPOINTS & GRADIENT ACCUMULATION
# ════════════════════════════════════════════════════════════════════════════

if CONFIG['finetune_strategy'] != 'frozen':
    print("="*70)
    print("FINE-TUNING ENCODERS")
    print("="*70)
    
    grad_accum = CONFIG.get('gradient_accumulation', 1)
    effective_batch = CONFIG['batch_size'] * grad_accum
    print(f"Batch size: {CONFIG['batch_size']} x {grad_accum} accumulation = {effective_batch} effective")
    
    # Dataset
    class FTDataset(Dataset):
        def __init__(self, dataframe):
            self.data = dataframe.reset_index(drop=True)
        def __len__(self):
            return len(self.data)
        def __getitem__(self, idx):
            row = self.data.iloc[idx]
            return {
                'ab_seq': row['antibody_sequence'],
                'ag_seq': row['antigen_sequence'],
                'pKd': torch.tensor(row['pKd'], dtype=torch.float32)
            }
    
    def ft_collate(batch):
        return {
            'ab_seq': [b['ab_seq'] for b in batch],
            'ag_seq': [b['ag_seq'] for b in batch],
            'pKd': torch.stack([b['pKd'] for b in batch])
        }
    
    # Simple head
    class SimpleHead(nn.Module):
        def __init__(self, input_dim):
            super().__init__()
            self.head = nn.Sequential(
                nn.Linear(input_dim, 256),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(256, 1)
            )
        def forward(self, x):
            return self.head(x).squeeze(-1)
    
    head = SimpleHead(ab_output_dim + ag_output_dim).to(device).to(torch.float32)
    
    # Optimizer
    params = [
        {'params': [p for p in ab_model.parameters() if p.requires_grad], 'lr': CONFIG['ft_lr']},
        {'params': [p for p in ag_model.parameters() if p.requires_grad], 'lr': CONFIG['ft_lr']},
        {'params': head.parameters(), 'lr': CONFIG['ft_lr'] * 10},
    ]
    optimizer = torch.optim.AdamW(params, weight_decay=0.01)
    criterion = nn.MSELoss()
    
    # DataLoaders
    train_loader = DataLoader(FTDataset(train_df), batch_size=CONFIG['batch_size'],
                              shuffle=True, collate_fn=ft_collate, num_workers=2, pin_memory=True)
    val_loader = DataLoader(FTDataset(val_df), batch_size=CONFIG['batch_size'],
                            shuffle=False, collate_fn=ft_collate, num_workers=2, pin_memory=True)
    
    # Logger
    logger = TrainingLogger()
    
    # Try to resume
    start_epoch = 0
    start_batch = 0
    if CONFIG['resume_from_checkpoint']:
        ckpt = ckpt_manager.load_training_checkpoint(ab_model, ag_model, optimizer)
        if ckpt:
            start_epoch = ckpt['epoch']
            start_batch = ckpt['batch_idx']
            logger.history = ckpt['train_history']
    
    # Training loop
    print(f"\nTraining for {CONFIG['ft_epochs']} epochs...")
    
    for epoch in range(start_epoch, CONFIG['ft_epochs']):
        ab_model.train()
        ag_model.train()
        head.train()
        logger.start_epoch()
        
        optimizer.zero_grad()  # Zero at start of accumulation
        
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), 
                    desc=f"Epoch {epoch+1}/{CONFIG['ft_epochs']}")
        
        for batch_idx, batch in pbar:
            # Skip if resuming
            if epoch == start_epoch and batch_idx < start_batch:
                continue
            
            # Tokenize
            ab_seqs = [" ".join(list(s)) for s in batch['ab_seq']]
            ag_seqs = [" ".join(list(s)) for s in batch['ag_seq']]
            
            ab_tokens = ab_tokenizer(ab_seqs, return_tensors='pt', padding=True,
                                     truncation=True, max_length=CONFIG['max_ab_length']).to(device)
            ag_tokens = ag_tokenizer(ag_seqs, return_tensors='pt', padding=True,
                                     truncation=True, max_length=CONFIG['max_ag_length']).to(device)
            targets = batch['pKd'].to(device)
            
            # Forward
            with torch.amp.autocast('cuda', dtype=torch.float16):
                ab_out = ab_model(**ab_tokens)
                ag_out = ag_model(**ag_tokens)
                
                if CONFIG['layer_strategy'] == 'multi':
                    ab_emb = extract_embeddings_multilayer(ab_out.hidden_states, ab_tokens['attention_mask'],
                                                          CONFIG['multi_layers'], CONFIG['pooling'])
                    ag_emb = extract_embeddings_multilayer(ag_out.hidden_states, ag_tokens['attention_mask'],
                                                          CONFIG['multi_layers'], CONFIG['pooling'])
                elif CONFIG['layer_strategy'] == 'weighted_all':
                    ab_emb = extract_embeddings_weighted(ab_out.hidden_states, ab_tokens['attention_mask'], CONFIG['pooling'])
                    ag_emb = extract_embeddings_weighted(ag_out.hidden_states, ag_tokens['attention_mask'], CONFIG['pooling'])
                else:
                    ab_hidden = ab_out.last_hidden_state
                    ag_hidden = ag_out.last_hidden_state
                    mask_ab = ab_tokens['attention_mask'].unsqueeze(-1).float()
                    mask_ag = ag_tokens['attention_mask'].unsqueeze(-1).float()
                    ab_emb = (ab_hidden * mask_ab).sum(1) / mask_ab.sum(1)
                    ag_emb = (ag_hidden * mask_ag).sum(1) / mask_ag.sum(1)
                
                combined = torch.cat([ab_emb, ag_emb], dim=-1).float()
                pred = head(combined)
                loss = criterion(pred, targets) / grad_accum  # Scale loss
            
            loss.backward()
            logger.log_batch(loss.item() * grad_accum)  # Log unscaled loss
            
            # Gradient accumulation step
            if (batch_idx + 1) % grad_accum == 0:
                torch.nn.utils.clip_grad_norm_(
                    list(ab_model.parameters()) + list(ag_model.parameters()) + list(head.parameters()), 1.0
                )
                optimizer.step()
                optimizer.zero_grad()
            
            # Update progress
            pbar.set_postfix({
                'loss': f'{loss.item() * grad_accum:.4f}', 
                'avg': f'{np.mean(logger.current_epoch_losses[-50:]):.4f}'
            })
            
            # Checkpoint
            if (batch_idx + 1) % CONFIG['checkpoint_every_n_batches'] == 0:
                ckpt_manager.save_training_checkpoint(
                    epoch, batch_idx, ab_model, ag_model, optimizer, 
                    logger.history, {'train_loss': np.mean(logger.current_epoch_losses)}
                )
            
            # Memory management
            if (batch_idx + 1) % CONFIG['clear_cache_every_n_batches'] == 0:
                monitor.clear_memory()
        
        # Final optimizer step if not aligned with accumulation
        if len(train_loader) % grad_accum != 0:
            optimizer.step()
            optimizer.zero_grad()
        
        # Validation
        ab_model.eval()
        ag_model.eval()
        head.eval()
        
        val_preds, val_labels = [], []
        val_loss_sum = 0
        
        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating", leave=False):
                ab_seqs = [" ".join(list(s)) for s in batch['ab_seq']]
                ag_seqs = [" ".join(list(s)) for s in batch['ag_seq']]
                
                ab_tokens = ab_tokenizer(ab_seqs, return_tensors='pt', padding=True,
                                         truncation=True, max_length=CONFIG['max_ab_length']).to(device)
                ag_tokens = ag_tokenizer(ag_seqs, return_tensors='pt', padding=True,
                                         truncation=True, max_length=CONFIG['max_ag_length']).to(device)
                targets = batch['pKd'].to(device)
                
                with torch.amp.autocast('cuda', dtype=torch.float16):
                    ab_out = ab_model(**ab_tokens)
                    ag_out = ag_model(**ag_tokens)
                    
                    if CONFIG['layer_strategy'] == 'multi':
                        ab_emb = extract_embeddings_multilayer(ab_out.hidden_states, ab_tokens['attention_mask'],
                                                              CONFIG['multi_layers'], CONFIG['pooling'])
                        ag_emb = extract_embeddings_multilayer(ag_out.hidden_states, ag_tokens['attention_mask'],
                                                              CONFIG['multi_layers'], CONFIG['pooling'])
                    elif CONFIG['layer_strategy'] == 'weighted_all':
                        ab_emb = extract_embeddings_weighted(ab_out.hidden_states, ab_tokens['attention_mask'], CONFIG['pooling'])
                        ag_emb = extract_embeddings_weighted(ag_out.hidden_states, ag_tokens['attention_mask'], CONFIG['pooling'])
                    else:
                        ab_hidden = ab_out.last_hidden_state
                        ag_hidden = ag_out.last_hidden_state
                        mask_ab = ab_tokens['attention_mask'].unsqueeze(-1).float()
                        mask_ag = ag_tokens['attention_mask'].unsqueeze(-1).float()
                        ab_emb = (ab_hidden * mask_ab).sum(1) / mask_ab.sum(1)
                        ag_emb = (ag_hidden * mask_ag).sum(1) / mask_ag.sum(1)
                    
                    combined = torch.cat([ab_emb, ag_emb], dim=-1).float()
                    pred = head(combined)
                    val_loss_sum += criterion(pred, targets).item()
                
                val_preds.extend(pred.cpu().numpy())
                val_labels.extend(targets.cpu().numpy())
        
        val_preds = np.array(val_preds)
        val_labels = np.array(val_labels)
        val_loss = val_loss_sum / len(val_loader)
        val_r2 = r2_score(val_labels, val_preds)
        val_mae = mean_absolute_error(val_labels, val_preds)
        
        logger.end_epoch(val_loss, val_r2, val_mae)
        logger.print_epoch_summary(epoch)
        monitor.print_status("Resources: ")
        
        # Epoch checkpoint
        if (epoch + 1) % CONFIG['checkpoint_every_n_epochs'] == 0:
            ckpt_manager.save_training_checkpoint(
                epoch + 1, 0, ab_model, ag_model, optimizer,
                logger.history, {'val_loss': val_loss, 'val_r2': val_r2, 'val_mae': val_mae}
            )
        
        start_batch = 0  # Reset for next epoch
    
    print("\n" + "="*70)
    print("FINE-TUNING COMPLETE!")
    print("="*70)
    print(f"Best Val R²: {max(logger.history['val_r2']):.4f}")
    print(f"Best Val MAE: {min(logger.history['val_mae']):.4f}")
    
    # Cleanup
    del head, optimizer
    monitor.clear_memory()
    
else:
    print("Strategy is FROZEN - skipping fine-tuning")

# Set to eval mode
ab_model.eval()
ag_model.eval()

## Generate Embeddings (with checkpoints)

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# GENERATE EMBEDDINGS WITH CHECKPOINTS
# ════════════════════════════════════════════════════════════════════════════

print("="*70)
print("GENERATING EMBEDDINGS")
print("="*70)

n_samples = len(df)
batch_size = CONFIG['batch_size'] * 2  # Larger for inference
labels = df['pKd'].values.astype(np.float32)

print(f"Samples: {n_samples:,}")
print(f"Batch size: {batch_size}")
print(f"Layer strategy: {CONFIG['layer_strategy']}")

# Try to resume
start_idx = 0
if CONFIG['resume_from_checkpoint']:
    ckpt = ckpt_manager.load_embedding_checkpoint()
    if ckpt:
        start_idx = ckpt['processed_idx']
        ab_embeddings = np.zeros((n_samples, ab_output_dim), dtype=np.float32)
        ag_embeddings = np.zeros((n_samples, ag_output_dim), dtype=np.float32)
        ab_embeddings[:start_idx] = ckpt['ab_embeddings']
        ag_embeddings[:start_idx] = ckpt['ag_embeddings']
    else:
        ab_embeddings = np.zeros((n_samples, ab_output_dim), dtype=np.float32)
        ag_embeddings = np.zeros((n_samples, ag_output_dim), dtype=np.float32)
else:
    ab_embeddings = np.zeros((n_samples, ab_output_dim), dtype=np.float32)
    ag_embeddings = np.zeros((n_samples, ag_output_dim), dtype=np.float32)

print(f"Starting from index: {start_idx:,}")

start_time = time.time()
batches_since_checkpoint = 0

pbar = tqdm(range(start_idx, n_samples, batch_size), desc="Encoding", 
            initial=start_idx//batch_size, total=(n_samples + batch_size - 1)//batch_size)

for i in pbar:
    batch_end = min(i + batch_size, n_samples)
    batch_df = df.iloc[i:batch_end]
    
    # Prepare sequences
    ab_seqs = [" ".join(list(s)) for s in batch_df['antibody_sequence'].tolist()]
    ag_seqs = [" ".join(list(s)) for s in batch_df['antigen_sequence'].tolist()]
    
    # Tokenize
    ab_tokens = ab_tokenizer(ab_seqs, return_tensors='pt', padding=True,
                             truncation=True, max_length=CONFIG['max_ab_length']).to(device)
    ag_tokens = ag_tokenizer(ag_seqs, return_tensors='pt', padding=True,
                             truncation=True, max_length=CONFIG['max_ag_length']).to(device)
    
    # Encode
    with torch.no_grad(), torch.amp.autocast('cuda', dtype=torch.float16):
        ab_out = ab_model(**ab_tokens)
        ag_out = ag_model(**ag_tokens)
        
        if CONFIG['layer_strategy'] == 'multi':
            ab_emb = extract_embeddings_multilayer(ab_out.hidden_states, ab_tokens['attention_mask'],
                                                  CONFIG['multi_layers'], CONFIG['pooling'])
            ag_emb = extract_embeddings_multilayer(ag_out.hidden_states, ag_tokens['attention_mask'],
                                                  CONFIG['multi_layers'], CONFIG['pooling'])
        elif CONFIG['layer_strategy'] == 'weighted_all':
            ab_emb = extract_embeddings_weighted(ab_out.hidden_states, ab_tokens['attention_mask'], CONFIG['pooling'])
            ag_emb = extract_embeddings_weighted(ag_out.hidden_states, ag_tokens['attention_mask'], CONFIG['pooling'])
        else:
            ab_hidden = ab_out.last_hidden_state
            ag_hidden = ag_out.last_hidden_state
            mask_ab = ab_tokens['attention_mask'].unsqueeze(-1).float()
            mask_ag = ag_tokens['attention_mask'].unsqueeze(-1).float()
            ab_emb = (ab_hidden * mask_ab).sum(1) / mask_ab.sum(1)
            ag_emb = (ag_hidden * mask_ag).sum(1) / mask_ag.sum(1)
    
    ab_embeddings[i:batch_end] = ab_emb.float().cpu().numpy()
    ag_embeddings[i:batch_end] = ag_emb.float().cpu().numpy()
    
    batches_since_checkpoint += 1
    
    # Update progress
    elapsed = time.time() - start_time
    speed = (batch_end - start_idx) / elapsed if elapsed > 0 else 0
    eta = (n_samples - batch_end) / speed / 60 if speed > 0 else 0
    pbar.set_postfix({'speed': f'{speed:.0f}/s', 'ETA': f'{eta:.1f}min'})
    
    # Checkpoint
    if batches_since_checkpoint >= CONFIG['checkpoint_every_n_batches']:
        ckpt_manager.save_embedding_checkpoint(batch_end, ab_embeddings, ag_embeddings, labels)
        batches_since_checkpoint = 0
    
    # Memory management
    if (i // batch_size) % CONFIG['clear_cache_every_n_batches'] == 0:
        monitor.clear_memory()

elapsed = time.time() - start_time
print(f"\nCompleted in {elapsed/60:.1f} minutes")
print(f"Speed: {(n_samples - start_idx)/elapsed:.1f} samples/sec")
monitor.print_status("Final: ")

## Save Embeddings

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# SAVE EMBEDDINGS
# ════════════════════════════════════════════════════════════════════════════

print("="*70)
print("SAVING EMBEDDINGS")
print("="*70)

# Split into train/val/test
from sklearn.model_selection import train_test_split

indices = np.arange(n_samples)
train_idx, temp_idx = train_test_split(indices, test_size=0.2, random_state=42)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42)

print(f"Train: {len(train_idx):,} | Val: {len(val_idx):,} | Test: {len(test_idx):,}")

# Config string for filename
config_str = ckpt_manager.config_str

# Save train
train_file = os.path.join(OUTPUT_DIR, f"train_embeddings_{config_str}.npz")
np.savez_compressed(
    train_file,
    antibody_embeddings=ab_embeddings[train_idx],
    antigen_embeddings=ag_embeddings[train_idx],
    labels=labels[train_idx],
)
print(f"✅ Train saved: {train_file}")

# Save val
val_file = os.path.join(OUTPUT_DIR, f"val_embeddings_{config_str}.npz")
np.savez_compressed(
    val_file,
    antibody_embeddings=ab_embeddings[val_idx],
    antigen_embeddings=ag_embeddings[val_idx],
    labels=labels[val_idx],
)
print(f"✅ Val saved: {val_file}")

# Save test
test_file = os.path.join(OUTPUT_DIR, f"test_embeddings_{config_str}.npz")
np.savez_compressed(
    test_file,
    antibody_embeddings=ab_embeddings[test_idx],
    antigen_embeddings=ag_embeddings[test_idx],
    labels=labels[test_idx],
)
print(f"✅ Test saved: {test_file}")

# Save metadata
metadata = {
    'config': CONFIG,
    'n_samples': n_samples,
    'n_train': len(train_idx),
    'n_val': len(val_idx),
    'n_test': len(test_idx),
    'ab_dim': ab_output_dim,
    'ag_dim': ag_output_dim,
    'total_dim': ab_output_dim + ag_output_dim,
    'generation_time_minutes': elapsed / 60,
    'timestamp': datetime.now().isoformat(),
}

metadata_file = os.path.join(OUTPUT_DIR, f"metadata_{config_str}.json")
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2, default=str)
print(f"✅ Metadata saved: {metadata_file}")

# Save LoRA weights if applicable
if CONFIG['finetune_strategy'] == 'lora':
    lora_dir = os.path.join(OUTPUT_DIR, f"lora_weights_{config_str}")
    os.makedirs(lora_dir, exist_ok=True)
    ab_model.save_pretrained(os.path.join(lora_dir, 'antibody'))
    ag_model.save_pretrained(os.path.join(lora_dir, 'antigen'))
    print(f"✅ LoRA weights saved: {lora_dir}")

# File sizes
print(f"\nFile sizes:")
for f in [train_file, val_file, test_file]:
    size = os.path.getsize(f) / (1024**3)
    print(f"  {os.path.basename(f)}: {size:.2f} GB")

# Clear checkpoints (completed successfully)
print("\nClearing checkpoints...")
ckpt_manager.clear_checkpoints()

print("\n" + "="*70)
print("STAGE 1 COMPLETE!")
print("="*70)
print(f"\nNext: Open Stage2_Fast_Training.ipynb")
print(f"Set embeddings_dir to: {OUTPUT_DIR}")
print(f"Use files with suffix: {config_str}")

In [ ]:
# Cleanup
del ab_model, ag_model, ab_embeddings, ag_embeddings
monitor.clear_memory()
print("Memory cleaned up!")
monitor.print_status()